In [ ]:
Importing necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb

from google.colab import files

uploaded = files.upload()

Saving trainWiDS.csv to trainWiDS.csv


In [ ]:
import io
#import csv
df = pd.read_csv(io.BytesIO(uploaded['trainWiDS.csv']))
#df = pd.read_csv('trainWiDS.csv', quoting=csv.QUOTE_NONE)
df.set_index("id", inplace = True)
df.head()

,title,author,text,label
id,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.isnull().sum()

title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
print(df.shape)
dropped_df = df.dropna(subset=['text'])
dropped_df.shape

(20800, 4)


(20761, 4)

In [ ]:
true_news = dropped_df[dropped_df['label'] == 1]
print(true_news.shape)
false_news = dropped_df[dropped_df['label'] == 0]
print(false_news.shape)

(10374, 4)
(10387, 4)


In [ ]:
from collections import Counter
from tqdm import tqdm
import re
import nltk
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
dropped_df.isnull().sum()

title      558
author    1918
text         0
label        0
dtype: int64

In [ ]:
print(dropped_df.author.unique().size)
dropped_df = dropped_df.fillna('')

4202


In [ ]:
def preprocess_text(text_data):
    preprocessed_text = []

    for sentence in tqdm(text_data):
        sentence = re.sub(r'[^\w\s]', '', sentence)
        preprocessed_text.append(' '.join(token.lower()
                                  for token in str(sentence).split()
                                  if token not in stopwords.words('english')))

    return preprocessed_text


In [ ]:
preprocessed_df = preprocess_text(dropped_df['text'].values)
preprocessed_df

100%|██████████| 20761/20761 [30:25<00:00, 11.37it/s]


['house dem aide we didnt even see comeys letter until jason chaffetz tweeted it by darrell lucus october 30 2016 subscribe jason chaffetz stump american fork utah image courtesy michael jolley available creative commonsby license with apologies keith olbermann doubt worst person the world weekfbi director james comey but according house democratic aide looks like also know secondworst person well it turns comey sent nowinfamous letter announcing fbi looking emails may related hillary clintons email server ranking democrats relevant committees didnt hear comey they found via tweet one republican committee chairmen as know comey notified republican chairmen democratic ranking members house intelligence judiciary oversight committees agency reviewing emails recently discovered order see contained classified information not long letter went oversight committee chairman jason chaffetz set political world ablaze tweet fbi dir informed the fbi learned existence emails appear pertinent invest

In [ ]:
dropped_df['text'] = preprocessed_df
dropped_df['text']

id
0        house dem aide we didnt even see comeys letter...
1        ever get feeling life circles roundabout rathe...
2        why truth might get you fired october 29 2016 ...
3        videos 15 civilians killed in single us airstr...
4        print an iranian woman sentenced six years pri...
                               ...                        
20795    rapper t i unloaded black celebrities met dona...
20796    when green bay packers lost washington redskin...
20797    the macys today grew union several great names...
20798    nato russia to hold parallel exercises in balk...
20799    david swanson author activist journalist radio...
Name: text, Length: 20761, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

train_X, val_X, train_y, val_y = train_test_split(dropped_df["title"], dropped_df["label"], random_state = 0, test_size=0.20)
print(train_X.shape)
print(train_y.shape)

(16608,)
(16608,)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
train_X = vectorization.fit_transform(train_X)
val_X = vectorization.transform(val_X)

In [ ]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics


model = LogisticRegression()
model.fit(train_X, train_y)

# testing the model accuracy
print("Accuracy: " + str(metrics.accuracy_score(val_y, model.predict(val_X))))

# testing the model precision
print("Precision: " + str(metrics.precision_score(val_y, model.predict(val_X))))

# testing the model recall
print("Recall: " + str(metrics.recall_score(val_y, model.predict(val_X))))

# testing the model f1 score
print("F1 Score: " + str(metrics.f1_score(val_y, model.predict(val_X))))

Accuracy: 0.9282446424271611
Precision: 0.8824814329401486
Recall: 0.9858467545143973
F1 Score: 0.9313047487321348


In [ ]:
# Decision Tree

from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(train_X, train_y)

# testing the model accuracy
print("Accuracy: " + str(metrics.accuracy_score(val_y, model.predict(val_X))))

# testing the model precision
print("Precision: " + str(metrics.precision_score(val_y, model.predict(val_X))))

# testing the model recall
print("Recall: " + str(metrics.recall_score(val_y, model.predict(val_X))))

# testing the model f1 score
print("F1 Score: " + str(metrics.f1_score(val_y, model.predict(val_X))))


Accuracy: 0.9162051529015169
Precision: 0.9090909090909091
Recall: 0.9224011713030746
F1 Score: 0.9156976744186045


In [ ]:
# Random Forest

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
model = RandomForestClassifier(n_estimators= 100, criterion="entropy")
model.fit(train_X, train_y)

# testing the model accuracy
print("Accuracy: " + str(metrics.accuracy_score(val_y, model.predict(val_X))))

# testing the model precision
print("Precision: " + str(metrics.precision_score(val_y, model.predict(val_X))))

# testing the model recall
print("Recall: " + str(metrics.recall_score(val_y, model.predict(val_X))))

# testing the model f1 score
print("F1 Score: " + str(metrics.f1_score(val_y, model.predict(val_X))))

Accuracy: 0.9340235973994703
Precision: 0.8904531456225253
Recall: 0.9877989263055149
F1 Score: 0.9366034243405831


In [ ]:
# Hyper parameter tuning for Random Forest

from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

parameters = {'n_estimators': randint(50,500),
              'max_depth': randint(1,20)}

model = RandomForestClassifier()

# Using random search to find the best possible parameters
rand_search = RandomizedSearchCV(model,
                                 param_distributions = parameters,
                                 n_iter=5,
                                 cv=5)

# Fit the random search object to the data
rand_search.fit(train_X, train_y)

best_param = rand_search.best_estimator_
print(rand_search.best_params_)

{'max_depth': 11, 'n_estimators': 249}


In [ ]:
# Random forest with best parameters

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
model = RandomForestClassifier(n_estimators= 249, criterion="entropy")
model.fit(train_X, train_y)

# testing the model accuracy
print("Accuracy: " + str(metrics.accuracy_score(val_y, model.predict(val_X))))

# testing the model precision
print("Precision: " + str(metrics.precision_score(val_y, model.predict(val_X))))

# testing the model recall
print("Recall: " + str(metrics.recall_score(val_y, model.predict(val_X))))

# testing the model f1 score
print("F1 Score: " + str(metrics.f1_score(val_y, model.predict(val_X))))

Accuracy: 0.9337828076089574
Precision: 0.8904049295774648
Recall: 0.9873108833577355
F1 Score: 0.9363573246933581


In [ ]:
# Naive bayes

from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
model = GaussianNB()
new_train_X = train_X.toarray()
new_val_X = val_X.toarray()
model.fit(new_train_X, train_y)
y_pred = model.predict(new_val_X)

# testing the model accuracy
print("Accuracy: " + str(metrics.accuracy_score(val_y, model.predict(new_val_X))))

# testing the model precision
print("Precision: " + str(metrics.precision_score(val_y, model.predict(new_val_X))))

# testing the model recall
print("Recall: " + str(metrics.recall_score(val_y, model.predict(new_val_X))))

# testing the model f1 score
print("F1 Score: " + str(metrics.f1_score(val_y, model.predict(new_val_X))))

Accuracy: 0.6491692752227306
Precision: 0.7560553633217993
Recall: 0.4265495363591996
F1 Score: 0.5453978159126365


In [ ]:
# Support Vector Machines

from sklearn.svm import SVC
model = SVC(kernel = "linear", random_state = 0)

model.fit(train_X, train_y)

from sklearn import metrics

# testing the accuracy of the model
SVM_accuracy = metrics.accuracy_score(val_y, model.predict(val_X))
print("accuracy = " + str(SVM_accuracy))

# testing the precision of the model
print("Precision: " + str(metrics.precision_score(val_y, model.predict(val_X))))

# testing the recall
print("Recall: " + str(metrics.recall_score(val_y, model.predict(val_X))))

# testing the f1 score
print("F1 Score: " + str(metrics.f1_score(val_y, model.predict(val_X))))

accuracy = 0.9378762340476764
Precision: 0.9024719101123595
Recall: 0.9799902391410444
F1 Score: 0.9396350023397285
